In [0]:
pip install kaggle

In [0]:
# Autenticação e uso da API Kaggle para listar arquivos
import os

KAGGLE_USERNAME = dbutils.secrets.get(scope="kaggle_scope", key="KAGGLE_USERNAME")
KAGGLE_KEY = dbutils.secrets.get(scope="kaggle_scope", key="KAGGLE_KEY")

os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
os.environ["KAGGLE_KEY"] = KAGGLE_KEY

In [0]:
# Liste os arquivos do dataset e obtenha as URLs diretas
# Use a API do Kaggle para obter a lista dos arquivos e os URLs de download:

from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

dataset = "teocalvo/teomewhy-loyalty-system"
files = api.dataset_list_files(dataset).files

file_links = []
for f in files:
    # A URL funciona com: https://www.kaggle.com/datasets/{owner_slug}/{dataset_slug}/download?file={filename}
    url = f"https://www.kaggle.com/datasets/{dataset}/download?file={f.name}"
    file_links.append({"filename": f.name, "url": url})

# Exemplo de como ficam os links
for item in file_links:
    print(item)


In [0]:
# Download dos arquivos via HTTP e salvando direto no volume S3/UC

import requests
from io import BytesIO
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

ACTUAL_PATH = "/Volumes/raw/data/actual"

def download_and_save_csv_kaggle(file_info):
    print(f"Baixando {file_info['filename']} de {file_info['url']} ...")
    response = requests.get(file_info['url'])
    response.raise_for_status()
    
    csv_bytes = BytesIO(response.content)
    # Ajuste o separador conforme seu dataset
    sep = ";" if "clientes" in file_info['filename'] else ","
    
    df = spark.read.csv(csv_bytes, header=True, sep=sep)

    output_path = f"{ACTUAL_PATH}/{file_info['filename']}"
    df.write.mode("overwrite").option("header", "true").csv(output_path)
    
    print(f"Arquivo salvo em {output_path}")

for item in file_links:
    download_and_save_csv_kaggle(item)
